## 准备数据

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

2025-03-21 23:13:56.516270: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742570036.532338 1262579 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742570036.539357 1262579 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1742570036.551233 1262579 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1742570036.551253 1262579 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1742570036.551255 1262579 computation_placer.cc:177] computation placer alr

In [2]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [3]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        self.W1 = tf.Variable(shape=[28*28, 100], dtype=tf.float32,
                         initial_value=tf.random.uniform(shape=[28*28, 100],
                                                         minval=-0.1, maxval=0.1))
        self.b1 = tf.Variable(shape=[100], dtype=tf.float32, initial_value=tf.zeros(100))
        self.W2 = tf.Variable(shape=[100, 10], dtype=tf.float32,
                         initial_value=tf.random.uniform(shape=[100, 10],
                                                         minval=-0.1, maxval=0.1))
        self.b2 = tf.Variable(shape=[10], dtype=tf.float32, initial_value=tf.zeros(10))
        ####################
    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        flat_x = tf.reshape(x, shape=[-1, 28*28])
        h1 = tf.tanh(tf.matmul(flat_x, self.W1) + self.b1)
        logits = tf.matmul(h1, self.W2) + self.b2
        ####################
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

W0000 00:00:1742570037.856091 1262579 gpu_device.cc:2341] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


## 计算 loss

In [4]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [5]:
train_data, test_data = mnist_dataset()
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 2.3629487 ; accuracy 0.094283335
epoch 1 : loss 2.357608 ; accuracy 0.097433336
epoch 2 : loss 2.352307 ; accuracy 0.10051667
epoch 3 : loss 2.3470447 ; accuracy 0.10355
epoch 4 : loss 2.3418198 ; accuracy 0.10706667
epoch 5 : loss 2.336631 ; accuracy 0.11075
epoch 6 : loss 2.3314774 ; accuracy 0.11385
epoch 7 : loss 2.326358 ; accuracy 0.1172
epoch 8 : loss 2.321272 ; accuracy 0.120733336
epoch 9 : loss 2.3162177 ; accuracy 0.124
epoch 10 : loss 2.3111951 ; accuracy 0.12766667
epoch 11 : loss 2.3062027 ; accuracy 0.13128333
epoch 12 : loss 2.3012395 ; accuracy 0.13498333
epoch 13 : loss 2.2963057 ; accuracy 0.1392
epoch 14 : loss 2.2913995 ; accuracy 0.14288333
epoch 15 : loss 2.2865202 ; accuracy 0.14693333
epoch 16 : loss 2.2816672 ; accuracy 0.15106666
epoch 17 : loss 2.2768402 ; accuracy 0.15528333
epoch 18 : loss 2.272038 ; accuracy 0.1593
epoch 19 : loss 2.26726 ; accuracy 0.16381666
epoch 20 : loss 2.2625055 ; accuracy 0.16761667
epoch 21 : loss 2.2577746 ; accur